In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("../creditcard.csv")
data = pd.DataFrame(data)

In [5]:
# data.isna().sum() = Conta a quantidade de valores NAN

n_transacoes = data['Class'].count()
n_fraudes = data['Class'].sum()
n_normais = n_transacoes - n_fraudes
fraudes_porc = n_fraudes / n_transacoes
nomais_porc = n_normais / n_transacoes

# "%.2f" %(fraudes_porc*100) = Deixa o resultado com 2 casas decimais

print("Número de transações: ", n_transacoes)
print("Número de fraudes: ", n_fraudes, "%.2f" %(fraudes_porc*100))
print("Número de transações normais", n_normais, "%.2f" %(nomais_porc*100))

Número de transações:  284807
Número de fraudes:  492 0.17
Número de transações normais 284315 99.83


In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

def executar_validador(x, y):
    validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)

    for treino_id , teste_id in validador.split(x, y):
        x_train, x_test = x[treino_id], x[teste_id]
        y_train, y_test = y[treino_id], y[teste_id]

    return x_train, x_test, y_train, y_test

In [7]:
%%time
from sklearn import tree

def executar_classificador(classificador, x_train, x_test, y_train):
    arvore = classificador.fit(x_train, y_train)
    y_pred = arvore.predict(x_test)
    return y_pred

CPU times: total: 219 ms
Wall time: 1.72 s


In [8]:
import matplotlib.pyplot as plt

def salvar_arvore(classificador, nome):
    plt.figure(figsize=(200,100))
    tree.plot_tree(classificador, filled=True, fontsize=14)
    plt.savefig(nome)
    plt.close()

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

def validar_arvore(y_test, y_pred):
    print(accuracy_score(y_test, y_pred)) # Calcula a proporção de previsões corretas em relação ao total de previsões feitas. è uma boa métrica geral, mas pode ser enganosa em datasets desbalanceados
    # precision_score e recall_score são utilizados em conjunto
    print(precision_score(y_test, y_pred)) # A precisão mede a proporção de verdadeiros positivos em relação ao total de positivos previstos (verdadeiros e falsos). Crucial em contextos onde os falsos positivos têm um custo alto
    print(recall_score(y_test, y_pred)) # Mede a proporção de verdadeiros positivos em relação ao total de reais positivos (verdadeiros positivos e falsos negativos). É importante em contextos onde é mais crítico identificar todas as ocorrências positivas
    print(confusion_matrix(y_test, y_pred)) # A matriz de confusão fornece uma visão detalhada das previsões do modelo, mostrando o número de verdadeiros positivos, falsos positivos, verdadeiros negativos e falsos negativos

In [10]:
# Execução do validador
x = data.drop('Class', axis=1).values
y = data['Class'].values

x_train, x_test, y_train, y_test = executar_validador(x, y)

In [11]:
# execução do classificador DecisionTreeClassifier
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

In [12]:
# Criação da figura da arvore de decisão
salvar_arvore(classificador_arvore_decisao, "arvore_decisao1.png")

In [23]:
# Validação árvore de decisão

validar_arvore(y_test, y_pred_arvore_decisao)

# [[ n° acertos (sem fraude) - VN, n° erros (sem fraude) - FP ][ n° erros (fraude) - FN, n° acertos (fraude) - VP ]]

0.9991924440855307
[[28421    12]
 [   11    37]]
0.7551020408163265
0.7708333333333334


In [26]:
print(classificador_arvore_decisao.get_depth())

21


In [ ]:
# execução do classificador DecisionTreeClassifier
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

In [29]:
validar_arvore(y_test, y_pred_arvore_decisao)

0.9994733330992591
[[28430     2]
 [   13    36]]
0.9473684210526315
0.7346938775510204


In [30]:
# execução do classificador DecisionTreeClassifier
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

In [31]:
validar_arvore(y_test, y_pred_arvore_decisao)

0.9993679997191109
[[28426     6]
 [   12    37]]
0.8604651162790697
0.7551020408163265


In [32]:
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=5, random_state=0)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

In [33]:
validar_arvore(y_test, y_pred_arvore_decisao)

0.999403110845827
[[28429     3]
 [   14    35]]
0.9210526315789473
0.7142857142857143


> Modelos Ensemble
> Bagging significa Bootstrapping + Aggregating
> Bootstramping sampling é quando divide um grupo de instâncias em amostras

> Random Forest, por exemplo, gera várias árvores aleatórias pegando informações de transações e seus atributos aleatóriamente

> Erro Out of Bag, forma de qualificar a Árvore Aleátoria 

In [35]:
from sklearn.ensemble import RandomForestClassifier

classificador_random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
y_pred_random_forest = executar_classificador(classificador_random_forest, x_train, x_test, y_train)

In [37]:
salvar_arvore(classificador_random_forest.estimators_[0], "random_forest1")
salvar_arvore(classificador_random_forest.estimators_[1], "random_forest2")

In [38]:
validar_arvore(y_test, y_pred_random_forest)

0.9995084442259752
0.9487179487179487
0.7551020408163265
[[28430     2]
 [   12    37]]


In [39]:
classificador_random_forest = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10)
y_pred_random_forest = executar_classificador(classificador_random_forest, x_train, x_test, y_train)

In [40]:
validar_arvore(y_test, y_pred_random_forest)

0.9995435553526912
0.9736842105263158
0.7551020408163265
[[28431     1]
 [   12    37]]


> Modelo Boosting
> - Modelos que se completam

> Algoritimo AdaBoost
> - Melhora os erros feitos na árvore gerada anteriormente

In [41]:
from sklearn.ensemble import AdaBoostClassifier

classificador_adaboost = AdaBoostClassifier(random_state=0)
y_pred_adabost = executar_classificador(classificador_adaboost, x_train, x_test, y_train)

c:\Users\Thiago Santos\Documents\coisasMinhas\Studies\data_science\envi\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [ ]:
salvar_arvore(classificador_adaboost.estimators_[0], "adab(oost1")
salvar_arvore(classificador_adaboost.estimators_[1], "adaboost2")
validar_arvore(y_test, y_pred_adabost)

In [ ]:
classificador_adaboost = AdaBoostClassifier(random_state=0, n_estimators=100)
y_pred_adabost = executar_classificador(classificador_adaboost, x_train, x_test, y_train)

In [42]:
classificador_adaboost = AdaBoostClassifier(random_state=0, n_estimators=200)
y_pred_adabost = executar_classificador(classificador_adaboost, x_train, x_test, y_train)

c:\Users\Thiago Santos\Documents\coisasMinhas\Studies\data_science\envi\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
validar_arvore(y_test, y_pred_adabost)